# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 09:51:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   45C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2021년 LG Data**

In [5]:
lexicon_2021 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2021_lg.csv')

lexicon_2021.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,LG화학,51910,매일경제,2021010104,[정유·화학] 최악국면 지났지만…석유제품 수요 변수,http://news.mk.co.kr/newsRead.php?no=391&year=...,◆ 신년기획 2021 경제기상도 ◆\r\n\n\n충남에 위치한 현대중공업 대산공장에...,2021-01-04,4,849000,900000,835000,889000,858451,0.078883,1,1,신년 기획 경제 기상도 충남 위치 현대 중공업 대산 공장 직원 업무 열중 모습 사진...


In [6]:
lexicon_2021[lexicon_2021['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2021.dropna(axis=0, inplace=True)
lexicon_2021 = lexicon_2021.reset_index(drop=True)
lexicon_2021.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2021 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2021['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2021['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2021['Tokenization'][x].split())):
                    if lexicon_2021['Tokenization'][x].split()[w] == list(set(lexicon_2021['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2021['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2021['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2021 = pos_lexicon_2021.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2021['date'][x], positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z])))

***2021-01-04 Cosine Similarity between <가치 있는> & <언택트> : 0.5496574640274048
***2021-01-04 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-01-04 Cosine Similarity between <도움이 되는> & <언택트> : 0.5378727912902832
***2021-01-04 Cosine Similarity between <의심의 여지가 없는> & <언택트> : 0.5293061137199402
***2021-01-04 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-01-04 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-04 Cosine Similarity between <호황> & <침체> : 0.6176011562347412
***2021-01-04 Cosine Similarity between <강세> & <급락> : 0.507910966873169
***2021-01-04 Cosine Similarity between <추천> & <추천> : 0.9999999403953552
***2021-01-04 Cosine Similarity between <해결> & <방법> : 0.5185782313346863
***2021-01-04 Cosine Similarity between <가치> & <의미> : 0.5354709625244141
***2021-01-04 Cosine Similarity between <강세> & <급락> : 0.507910966873169
***2021-01-04 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-01-04 Cosine Similarity betwe

In [11]:
pos_lexicon_2021

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2021-01-04,1,가치 있는,언택트,0.549657,1
1,2021-01-04,1,경신,기록,0.522991,1
2,2021-01-04,1,도움이 되는,언택트,0.537873,1
3,2021-01-04,1,의심의 여지가 없는,언택트,0.529306,1
4,2021-01-04,1,증대,증가,0.694643,4
...,...,...,...,...,...,...
9040,2021-09-16,1511,장점,특징,0.644235,1
9041,2021-09-16,1511,증대,증가,0.694643,1
9042,2021-09-16,1511,증대,확대,0.519324,2
9043,2021-09-16,1511,추진,검토,0.511331,1


In [12]:
pos_lexicon_2021.to_csv('../../../../Code/Data/Test/Stock-Year/pos_lg_2021.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2021['news_word'])))

a

['기가와트',
 '증가',
 '닛케이지수',
 '크래프톤크래프톤',
 '언택트주',
 '기가톤',
 '황경미',
 '전개',
 '밑돌',
 '변규칠',
 '팽팽하',
 '맞소송',
 '지박이',
 '가치나',
 '실패',
 '글루코시놀레이트',
 '소재효',
 '불황',
 '향상',
 '츠와츠',
 '인폴딩',
 '펀인터렉티브',
 '가장',
 '훈풍',
 '강력',
 '우호',
 '차익',
 '견조할',
 '요구',
 '언택트',
 '덴티움덴티움',
 '최진명',
 '퍼부으',
 '자이앤트',
 '배팅했다',
 '의미',
 '활기',
 '급락',
 '특성',
 '배정물량',
 '횡보할',
 '극복',
 '광풍',
 '진행',
 '스위스리',
 '최고점',
 '시너지',
 '증대',
 '업황',
 '백혜진',
 '이점',
 '증진',
 '최고급',
 '샤오펑모터스',
 '비밀보호법',
 '액티브펀드',
 '본격',
 '대차물량',
 '박기효',
 '게임스탑',
 '최길학',
 '감소',
 '강점',
 '검토',
 '이미드',
 '소형모듈원전',
 '일진일퇴',
 '머니무브',
 '겹악재',
 '디지털리서치',
 '짜릿거리',
 '시노텍스',
 '나정환',
 '문제해결',
 '자공시',
 '공유진',
 '설치량',
 '방안',
 '적극',
 '사업',
 '란저우시',
 '촉진',
 '문제',
 '스노우플레이크',
 '폭락',
 '팩토바',
 '저하',
 '들보',
 '침체',
 '레브친',
 '텍티앤에이텍',
 '듀비에정',
 '이노톡스',
 '펀드스트랫',
 '선우혜정',
 '간펑리튬',
 '방법',
 '알루코그룹',
 '렌탈식',
 '박병창',
 '기록',
 '확보',
 '추천',
 '확충',
 '단점',
 '확대',
 '확립',
 '류영재',
 '횡보세',
 '펜데믹',
 '텍오스코텍',
 '추진',
 '아달리무맙',
 '해법',
 '롱숏거래',
 '원격평',
 '중요',
 '락희화학',
 '활황',
 '맥쿼리인프라',
 '밑돈',
 '로컬밸류업',
 '

In [14]:
b = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.7]['news_word'])))

b

['추진', '최고점', '밑돈', '추천', '실패', '단점', '약세', '불황', '툴리눔']

In [15]:
c = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.65]['news_word'])))

c

['최길학',
 '추진',
 '증가',
 '감소',
 '최고점',
 '밑돈',
 '추천',
 '웃돌',
 '적극',
 '단점',
 '약세',
 '실패',
 '실린더헤드',
 '불황',
 '것이므',
 '빅테크주',
 '맞소송',
 '툴리눔']